# LAB 3

Networks and Systems Security
Week 03
Authentication and Access Control

### Setting up

In [ ]:
!pip install bcrypt pyotp qrcode pillow

### Section 1: Password Strength Analysis
Objective

To understand the fundamental criteria for a strong password and
how to programmatically analyse it.

Key Concepts
- Character Sets: The types of characters used (lowercase,
uppercase, numbers, symbols). Using a wider variety of character
sets dramatically increases entropy.
- Length: The single most important factor in password strength. An
8-character password is weak, 12 is good, and 16+ is excellent.

Code


In [7]:
import math
import string

with open("100k-most-used-passwords.txt", "r") as f:
    common_passwords = f.read().splitlines()

#function that give a password a strength score
def password_strength(password):
    score = 0
    pool_size = 0

    # length check
    if len(password) >= 8:
        score += 1
    if len(password) >= 12:
        score += 1

    # character variety
    if any(c.islower() for c in password):
        score += 1
        pool_size += 26
    if any(c.isupper() for c in password):
        score += 1
        pool_size += 26
    if any(c.isdigit() for c in password):
        score += 1
        pool_size += 10
    if any(1 for c in password if c in string.punctuation):
        score += 1
        pool_size += 32  

    #entropy 
    if pool_size > 0:
        entropy = len(password) * math.log2(pool_size)
        # print(f"Entropy: {entropy}")
        if entropy >= 25:
            score += 1
        if entropy >= 50:
            score += 1
        if entropy >= 75:
            score += 1
        if entropy >= 100:
            score += 1
       

    #common password check

    # common_passwords = {"password", "123456", "qwerty", "abc123"}

    if password.lower() in common_passwords:
        score = 0  

    return score

### testing ###
# passwords = [
#     "password",
#     "a", 
#     "A",
#     "Aa",
#     "Aa123",
#     "Aa123!",
#     "aaaaaaaaaaaaaaaaaaaaa",
#     "aAaAaAaAaAaaaAaAaAa",
#     "AaBbCc12345678£$%!",
#     "Pass123",
#     "MyP@ssw0rd"
# ]
# for pwd in passwords:
#     print(f"Password: {pwd}, Strength Score: {password_strength(pwd)}")
##################

password = input("Enter a password: ")
print(f"Password strength score (1-10): {password_strength(password)}")

Password strength score (1-10): 0



#### Output analysis
- Example 1

Input: Pass123

Output: Password strength score (1-10): 0

This gives it a score of 0 because the password has been found in the 100k common password list.

- Example 2

Input: MyP@ssw0rd

Output: Password strength score (1-10): 7

This gives us a higher strength score of 7, one point for length, four for character variety, and finally two for entropy.

#### Interpretation
Including a dictionary of breached passwords significantly improves real-world strength checking, along side this entropy score of ~100 means ~2^100 combinations, which is impractical for brute force attacks. Therfore the more unique and variety length a password has, the harder it is to crack

#### Limitations
This is however a very basic checker, limited only by length, variety, entropy and originality. This lack of checks for basic patterns means a password like "aAaAaAaAaAaaaAaAaAa" still scores increadibly high (Password strength score (1-10): 8) thanks so its length and case variety giving it a high entropy score.

#### Mini reflection
A lot more goes into creating a strong password than I have initially thought. Having made many passoword verifications in the past, sanitising inputs and creating regex patterns, not much thought was ever put into how these additions actually help to improve security from attackers. Cross checking with the common password list especially shows this, with passwords that would pass standard verification still scoring 0.


### Section 2: Don't Store Passwords, Store Hashes!

Objective

To understand the critical concept of password hashing and to
compare insecure hashing algorithms (MD5, SHA-256) with a
secure one (bcrypt).

Key concepts
- Hashing: A one-way cryptographic function that transforms an
input (like a password) into a fixed-size string of characters called
a hash. It's designed to be irreversible.
- MD5 & SHA-256: Fast hashing algorithms. Their speed is great for
checking file integrity but terrible for passwords, as it allows
attackers to guess billions of passwords per second.
- bcrypt: A password hashing function that is intentionally slow and
adaptive. You can increase the "work factor" (or rounds) over time
as computers get faster, keeping your hashes secure.

code


In [5]:
import hashlib
import bcrypt

password = "MyPassw0rd123!"

##hashing passwords##

#using md5
print("MD5:     ", hashlib.md5(password.encode()).hexdigest())

#using sha-256
print("SHA-256: ", hashlib.sha256(password.encode()).hexdigest())

#using bcrypt
salt = bcrypt.gensalt()
hashed = bcrypt.hashpw(password.encode(), salt)
print("Bcrypt:  ", hashed)


MD5:      e60abbe4f45ad3999e51f09785a30b4f
SHA-256:  02891c9977283692a20c6cde56bf9ba8fa2e3fe023b96e6dee32b89467739cac
Bcrypt:   b'$2b$12$nNb17hxoz.6UOqA78Jpr4uE/vJ6KuUlAgF/bId.tJpZgJSIMPchni'


#### Output analysis

- MD5 produces a short 32-character hex digest

- SHA-256 produces a 64-character digest

- bcrypt output contains additional metadata: algorithm version, work factor, salt, and hash

#### Security analysis

Due to their speed and lack of salt, MD5 and SHA-256 are very insecure despite being hashes

- an attacker with a modern GPU can try billions of guesses per second

- if two users choose the same password, they produce identical hashes 

This makes them vulnerable to brute force attacks and rainbow tables.

bcrypt compared is slow, adaprive, and automatically salted. This prevents both brute force attacks and rainbow tables, further increased with work factor over time, making it much more suitable for password storage.

In a real life scenario, if there was a databreach for passwords hashed with MD5, attackers could easily crack millions of these quickly. The same attack on bcrypt hashes becomes computationally expensive, potentially taking years depending on cost factor and password complexity.

#### Mini reflection
User expirience wise, slowing down the password creation process can be very annoying, but compared to how much longer these will then take to be cracked by attackers, it seems well worth it. I wasnt very aware of this before.

### Section 3: Adding Salt and Pepper
Objective

To understand how salting defeats pre-computation attacks (like
rainbow tables) and how a pepper adds another layer of security.

Key Concepts
- Rainbow Table: A precomputed table of hashes for common
passwords. If you don't use a salt, an attacker can find a matching
hash in their table and instantly know the password.
- Salt: A unique, random string that is added to a password before
hashing. Every user gets a different salt. It is stored in the
database alongside the hashed password.
- Pepper: A secret, static string that is added to a password before
hashing. Unlike a salt, the pepper is the same for all users and is
NOT stored in the database. It's stored securely in the application's
configuration or an environment variable.

code

In [6]:
import hashlib
import bcrypt

password = "MyPassw0rd123!"

#hashing with no salt produces same hash for same password
print("first    SHA-256 (no salt):              ", hashlib.sha256(password.encode()).hexdigest())
print("second   SHA-256 (no salt):              ", hashlib.sha256(password.encode()).hexdigest())

#hashing with salt produces different hash for same password
print("first    SHA-256 (with salt):            ", hashlib.sha256((password + bcrypt.gensalt().decode()).encode()).hexdigest())
print("second   SHA-256 (with salt):            ", hashlib.sha256((password + bcrypt.gensalt().decode()).encode()).hexdigest())

#adding pepper
pepper = "PEPPER!"
print("first    SHA-256 (with salt and pepper): ", hashlib.sha256((password + bcrypt.gensalt().decode() + pepper).encode()).hexdigest())
print("second   SHA-256 (with salt and pepper): ", hashlib.sha256((password + bcrypt.gensalt().decode() + pepper).encode()).hexdigest())


first    SHA-256 (no salt):               02891c9977283692a20c6cde56bf9ba8fa2e3fe023b96e6dee32b89467739cac
second   SHA-256 (no salt):               02891c9977283692a20c6cde56bf9ba8fa2e3fe023b96e6dee32b89467739cac
first    SHA-256 (with salt):             97cf90fd37146158fbd866323048aee858d347c38c38e4d30e56eccd61e8002c
second   SHA-256 (with salt):             d3b24ef4a5b8d726ca6ea2879edcc191018e7982777d3b2bfea4a88d18f914d1
first    SHA-256 (with salt and pepper):  2d8a09f2a39f6540ce407543adbba23eec96f53848c7cf117c282475fd439c6e
second   SHA-256 (with salt and pepper):  f1fa11893c717258f99417717fcff30167893e6cbd160bbcebf840df82d6edfa


#### Output analysis
- Without salt the same input results in the same output.
- With salt the same inputs have different salts, which means they have different outputs.
- With salt and pepper attackers are forced to compromise two systems (database + config storage)

#### Security analysis
If a database containing hashed passwords is compromised

- Without salt all passwords are in risk of being cracked with ranbow tables and brute force attacks.
- With salt all passwords can immedietely start to be cracked by attackers, but rainbow tables are now completly useless, an attacker now needing a unique table for every possible salt.
- With salt and pepper attackers must also compromise the server environment to retrieve the pepper before cracking, creating extra defense in depth since the database breach alone is no longer enough.

#### Mini reflection

I found the idea of a pepper especially interesting because it introduces system architecture considerations, not just cryptography, making the database alone insufficient for attackers to succeed. "Defense in depth"



### Section 4: Implementing Two-Factor Authentication (2FA)
Objective

To add a second layer of security using Time-based One-Time
Passwords (TOTP).

Key Concepts
- 2FA (Two-Factor Authentication): A security process where
users provide two different authentication factors. This is typically
"something you know" (password) and "something you have" (your
phone).
- TOTP (Time-based One-Time Password): An algorithm that uses
a shared secret key and the current time to generate a temporary,
single-use code.
- Provisioning URI: A special link (often shown as a QR code) that
contains the secret key, account name, and issuer information
needed for an authenticator app (like Google Authenticator or
Authy) to start generating codes.

Working example

Scan the following barcode with your phone’s OTP app (e.g. Google Authenticator):

![qrcode.png](qr.webp)

Now run the following and compare the output:


In [3]:
import pyotp
totp = pyotp.TOTP("JBSWY3DPEHPK3PXP")
print("Current OTP:", totp.now())

Current OTP: 603788


Using the current OTP allows you to verify your identity without having to use a proper password. 

code

In [ ]:
import pyotp
import time

#one time password generation

def generate_otp():
    totp = pyotp.TOTP(pyotp.random_base32())  
    print("Current OTP:", totp.now())

    user_otp = input("Enter OTP to verify: ")

    if totp.verify(user_otp):
        print("OTP "+user_otp+ " is valid!")
    else:
        print("OTP "+user_otp+ " is invalid!")

    time.sleep(10)

generate_otp()
generate_otp()


Current OTP: 768467
OTP 768467 is valid!
Current OTP: 322425
OTP 768467 is invalid!


#### Output analysis
Current OTP refreshes with every verification.

#### Security analysis
This demonstrates TOTP being an effective user verification method instead of passwords. It mitigates password reuse, protects users from database leaks (no passwords stored), and would be completly independant from the application server. 

Compared 2FA codes sent via SMS can be intercepted with man in the middle attacks, phone numbers can be stolen with SIM swap attacks, and past codes get stored on the device. This reliance on telecom networks makes it a big security risk comparitively.

#### Mini reflection
The idea of "something you know", "something you have" and "something you are" is very interesting. Most sites always just use passwords, which as im learning more about security, doesnt seem to be the best way to go about it. The idea of using "something you have" like for totp seems a lot more secure now. Other than this I faced many issues when generating a qr code, scanning it not working at all with my phone. This is the reason I opted out of this entirely, making a simple input instead of the qr code, which conceptially works the same. To improve this Id need to link this to another device like a phone, to confirm "something you have" properly.


### Section 5: Simulating a Brute-Force Attack
Objective

To visually demonstrate why fast hashes are weak and why slow,
salted hashes are strong.

Key Concepts
- Brute-Force Attack: An attack where a threat actor tries every
possible password combination until they find the correct one.
- Dictionary Attack: A more efficient form of brute-force where the
attacker uses a list of common words and passwords (a
"dictionary" or "wordlist").

code

In [5]:
import hashlib
import time

#dictionary attack function

def dictionary_attack(target_hash, hash_type):
    start = time.time()
    with open("100k-most-used-passwords.txt", "r") as f:
        common_passwords = f.read().splitlines()
    found = False
    for pwd in common_passwords:
        if hash_type == "md5":
            hashed_pwd = hashlib.md5(pwd.encode()).hexdigest()
        elif hash_type == "sha256":
            hashed_pwd = hashlib.sha256(pwd.encode()).hexdigest()
        else:
            print("Unsupported hash type")
            return

        if hashed_pwd == target_hash:
            found = True
            print(f"Password found using {hash_type}: {pwd}")
            print(f"Time taken: {time.time() - start} seconds\n")
            return
    if not found:
        print(f"Password not found in using {hash_type}.")
        print(f"Time taken: {time.time() - start} seconds\n")
          
        
## testing ###
# passwords = [
#     "password",
#     "a", 
#     "A",
#     "Aa",
#     "Aa123",
#     "Aa123!",
#     "aaaaaaaaaaaaaaaaaaaaa",
#     "aAaAaAaAaAaaaAaAaAa",
#     "AaBbCc12345678£$%!",
#     "Pass123",
#     "MyP@ssw0rd"
# ]
# for pwd in passwords:
#     dictionary_attack(hashlib.md5(pwd.encode()).hexdigest(), "md5")
#     dictionary_attack(hashlib.sha256(pwd.encode()).hexdigest(), "sha256")
#################


password = input("Enter a password: ")

dictionary_attack(hashlib.md5(password.encode()).hexdigest(), "md5")
dictionary_attack(hashlib.sha256(password.encode()).hexdigest(), "sha256")

password = input("Enter a password: ")

dictionary_attack(hashlib.md5(password.encode()).hexdigest(), "md5")
dictionary_attack(hashlib.sha256(password.encode()).hexdigest(), "sha256")

Password found using md5: cheese
Time taken: 0.01793360710144043 seconds

Password found using sha256: cheese
Time taken: 0.0032885074615478516 seconds

Password not found in using md5.
Time taken: 0.11761140823364258 seconds

Password not found in using sha256.
Time taken: 0.08267450332641602 seconds



#### Output analysis
The attack completes in miliseconds, with md5 being slightly slower than sha256 by a tiny amount. If the password isnt in the 100k world list, the program completes the entire search still in miliseconds.

#### Security analysis
In a real life scenario, attackers would use common and leaked password lists which contain millions of previously breached passwords. Because algorithms like MD5 and SHA-256 are extremely fast, attackers can hash millions of guesses per second using GPU-powered cracking tools. Once a password hash database is stolen, a fast hash function effectively helps the attacker by allowing high-speed cracking.

This is why bcrypt is so good, by being intentially slow one hash takes much longer. This means multiple guesses become detrimental when cracking. When combined with per-user salts, bcrypt makes brute-forcing millions of accounts computationally impractical.

#### Mini reflection
The speed of how quickly attackers can crack passwords with the right tools is astonishing. Really puts into perspective what really happens when there is a major database breach, eg. linked in having their records leaked in 2024 must have had attackers on it like a fly to a flame. The code I produced also is increadibly simple, which can be concerning despite it being just an example. I have also tried to simulate a proper brute force attack with a friend by trying to crack a 2 byte salt, however the computational time was so high we had to give up. If i were to do this again, id try to optimise our brute force code, and try to crack a more secured hashed password using all of a hackers tools.

## Section 6: Building the Complete System
Objective

To integrate all the concepts, we've discussed into a single, secure
authentication class.

Key Takeaways & Best Practices

This workshop covered the foundational pillars of modern, secure
authentication. Always remember:
- Enforce Strong Passwords: Use a password strength meter and
reject weak or common passwords.
- NEVER Store Plaintext Passwords: This is the #1 rule.
- Use a Slow, Adaptive Hash Function: Use bcrypt or a modern
alternative like Argon2. Avoid MD5 and SHA for passwords.
- Always Salt Passwords: Modern libraries like bcrypt do this
automatically. It protects you from rainbow table attacks.
- Implement 2FA: Offer TOTP as a standard security feature. It
protects users even if their password is stolen.
- Consider a Pepper: For an extra layer of defence, a system-wide
pepper can protect a compromised database.

code

In [21]:
import bcrypt
import pyotp

#database of users
users = {}
#pepper for password hashing
pepper = "PEPPER!"

def register_user(username, password):
        
    # password strength check (using previous function in section 1)
    if password_strength(password) < 5:
        print("User: "+username+", Password too weak. Registration denied.")
        return
        
    # hash the password
    hashed = bcrypt.hashpw((password+pepper).encode(), bcrypt.gensalt())

    # TOTP secret
    totp_instance = pyotp.random_base32()

    # Store in database
    users[username] = {
        "password_hash": hashed,
        "totp_instance": totp_instance
    }

    print("User: "+username+", Registration successful!")
    
def authenticate(username, password, otp):
    
    # check user exists
    if username not in users:
        print("User "+username+" not found.")
        return False

    # verify password (bcrypt)
    if not bcrypt.checkpw((password+pepper).encode(), users[username]["password_hash"]):
        print("Incorrect password for "+username+".")
        return False

    # verify TOTP
    totp = pyotp.TOTP(users[username]["totp_instance"])
    
    if not totp.verify(otp):
        print("Invalid otp for "+username+".")
        return False
    
    print("Login successful for "+username+"!")
    return True


# register users
print("REGISTRATION")
register_user("Cool Dude", "MyP@ssw0rd123!")
register_user("Hot Gal", "Pass123")

# authenticate testing
print("\nAUTHENTICATION ")
authenticate("Hot Gal", "MyP@ssw0rd123!", pyotp.TOTP(users["Cool Dude"]["totp_instance"]).now())
authenticate("Cool Dude", "Pass123", pyotp.TOTP(users["Cool Dude"]["totp_instance"]).now())
authenticate("Cool Dude", "MyP@ssw0rd123!", 123456)

# authenticate
authenticate("Cool Dude", "MyP@ssw0rd123!", pyotp.TOTP(users["Cool Dude"]["totp_instance"]).now())

REGISTRATION
User: Cool Dude, Registration successful!
User: Hot Gal, Password too weak. Registration denied.

AUTHENTICATION 
User Hot Gal not found.
Incorrect password for Cool Dude.
Invalid otp for Cool Dude.
Login successful for Cool Dude!


True

#### Security analysis

This code demonstrates strong password enforcement, adaptive hashing, database protection, totp requirenment, and overall defense in depth. 

If an attacker where to get access to the database, the stored hashed passwords and totps wouldnt be enough thanks to bcrypt significantly slowing down brute force attacks, and most importantly the pepper, which the db is useless without for the attacker (This usually stored in an .env).

#### Mini reflection
As someone that doesnt usually create and store users (primarily using auth libaries for this, entrusting their security eg. google), building a full authentication system helped me realise that security is not one control, but multiple layers working together. It was espessially interesting to see how simple the TOTP implementation is using pyotp, yet how powerful it is in reducing real-world risk, particularly against credential stuffing and password reuse attacks.
